## Let's Use Airbnb's API to get Data for Nashville Rentals

In [1]:
import requests
from bs4 import BeautifulSoup  
import pandas as pd

#### Make a request using the requests library

In [2]:
website_url = 'https://www.airbnb.com/s/East-Nashville--Nashville--TN--United-States/homes?refinement_paths%5B%5D=%2Fhomes&tab_id=home_tab&checkin=2020-12-18&checkout=2020-12-20&adults=10&search_type=filter_change&gps_lat=36.1005056&gps_lng=-86.7401728&price_max=510&place_id=ChIJQfjkuXtoZIgRnWBuVONAPJs'
request = requests.get(website_url)
request.status_code

200

#### Use [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) package to extract data from html docs (and xml docs)

In [3]:
bnb = BeautifulSoup(request.content, 'lxml')
print(bnb.title)

<title>Airbnb® | East Nashville, Nashville - Vacation Rentals &amp; Places to Stay - Tennessee, United States</title>


'300'

In [4]:
listings = bnb.find_all('div', attrs = {'class': '_8ssblpx'}) # listing level information

#### Let's put all of the listing information for the first page into a dataframe

In [5]:
names, night_cost, categories, suphost, urls, guests, total, reviews = [], [], [], [], [], [], [], []

for num in range(0, 20): 
    names.append(listings[num].find("div", {"class":"_bzh5lkq"}).text) #name of Airbnb
    night_cost.append(listings[num].find("div", {"class":"_1fwiw8gv"}).text) # cost per night stay
    categories.append(listings[num].find("div", {"class":"_1tanv1h"}).text) # what type of rental is this (i.e. 'Entire Home in Nashville')
    suphost.append(listings[num].find("div", {"class":"_1nz9l7j"}).text) # Superhost status
    urls.append(listings[num].find("div", {"class":"_1048zci"}).find('a')['href']) # url for the listing
    guests.append(listings[num].find("div", {"class":"_kqh46o"}).text) # guests and BR/BA
    reviews.append(listings[num].find("div", {"class":"_1bbeetd"}).text) # review count and avg star rating
    total.append(listings[num].find("div", {"class":"_vsjqit"}).text) # total cost for trip 
    
dictionary = {'name': names, 'nightly cost': night_cost, 'category': categories, 'superhost': suphost, 'url': urls, 'guests_rental': guests, 'total cost': total, 'reviews': reviews}
page_data = pd.DataFrame(dictionary)
page_data.head()

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews
0,⭐️Lovely East Nashville•Covered Deck with TV•3...,Previous price:$215Discounted price:$161 / night,Entire house in Nashville,SUPERHOST,/rooms/31877722?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 5 beds · 2 baths,Price:$464 totalShow details,Rating 5.0 out of 5;5.054 reviews (54)Previous...
1,"LUXURY MODERN HOME, GREAT FOR A STAYCATION W/ ...",Previous price:$267Discounted price:$189 / night,Entire townhouse in Nashville,SUPERHOST,/rooms/40539348?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 3 baths,Price:$603 totalShow details,Rating 4.93 out of 5;4.9314 reviews (14)Previo...
2,Butterfly BNB~Modern Luxury~Giant Rooftop~9Min>DT,Previous price:$591Discounted price:$356 / night,Entire house in Nashville,SUPERHOST,/rooms/40420743?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 11 beds · 3.5 baths,"Price:$1,212 totalShow details",Rating 4.92 out of 5;4.9237 reviews (37)Previo...
3,Nashville Rockin' Rollin' Getaway! SLEEPS 12!,Previous price:$198Discounted price:$163 / night,Entire townhouse in Nashville,,/rooms/45202865?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 5 beds · 3.5 baths,Price:$733 totalShow details,Previous price:$198Discounted price:$163 / nig...
4,*NEW* Panoramic 4BR/2.5BA City Retreat,Price:$169 / night,Entire house in Nashville,,/rooms/46511010?adults=10&check_in=2020-12-18&...,10 guests · 4 bedrooms · 5 beds · 2.5 baths,Price:$728 totalShow details,Price:$169 / nightPrice:$728 totalShow details


#### Iterate through the rest of the pages and pull the listing data

In [6]:
urls = []
names, night_cost, categories, suphost, list_urls, guests, total, reviews = [], [], [], [], [], [], [], []

for i in range(1, 15):
    url = website_url + '&items_offset=' + str(i * 20) + '&section_offset=3'
    urls.append(url)
    i = i+1

for j in range(0, 14):
    page = urls[j]
    response2 = requests.get(page)
    bnb2 = BeautifulSoup(response2.content, 'lxml')
    listings2 = bnb2.find_all('div', attrs = {'class': '_8ssblpx'})
    
    for num2 in range(0, 20): 
        names.append(listings2[num2].find("div", {"class":"_bzh5lkq"}).text) #name of Airbnb
        night_cost.append(listings2[num2].find("div", {"class":"_1fwiw8gv"}).text) # cost per night stay
        categories.append(listings2[num2].find("div", {"class":"_1tanv1h"}).text) # what type of rental is this (i.e. 'Entire Home in Nashville')
        suphost.append(listings2[num2].find("div", {"class":"_1nz9l7j"}).text) # Superhost status
        list_urls.append(listings2[num2].find("div", {"class":"_1048zci"}).find('a')['href']) # url for the listing
        guests.append(listings2[num2].find("div", {"class":"_kqh46o"}).text) # guests and BR/BA
        reviews.append(listings2[num2].find("div", {"class":"_1bbeetd"}).text) # review count and avg star rating
        total.append(listings2[num2].find("div", {"class":"_vsjqit"}).text) # total cost for trip 

dictionary = {'name': names, 'nightly cost': night_cost, 'category': categories, 'superhost': suphost, 'url': list_urls, 'guests_rental': guests, 'total cost': total, 'reviews': reviews}
search_data = pd.DataFrame(dictionary)

search_data

,name,nightly cost,category,superhost,url,guests_rental,total cost,reviews
0,"East Nashville Home, walk to dining & greenway",Price:$275 / night,Entire house in Nashville,,/rooms/45609934?adults=10&check_in=2020-12-18&...,12 guests · 5 bedrooms · 8 beds · 3 baths,Price:$787 totalShow details,Price:$275 / nightPrice:$787 totalShow details
1,"LUXURY MODERN HOME, SAFE GATHERING PLACE W/ BB...",Previous price:$262Discounted price:$189 / night,Entire townhouse in Nashville,SUPERHOST,/rooms/40539309?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 3 baths,Price:$603 totalShow details,Rating 4.89 out of 5;4.8918 reviews (18)Previo...
2,Nashville Rockin' Rollin' Getaway! SLEEPS 12!,Previous price:$198Discounted price:$163 / night,Entire townhouse in Nashville,,/rooms/45202865?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 5 beds · 3.5 baths,Price:$733 totalShow details,Previous price:$198Discounted price:$163 / nig...
3,4BR from Thomas Rhett Video✦Patio/Deck/Fenced ...,Previous price:$500Discounted price:$365 / night,Entire house in Nashville,SUPERHOST,/rooms/14088893?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 7 beds · 3 baths,Price:$947 totalShow details,Rating 4.96 out of 5;4.9649 reviews (49)Previo...
4,Jones ave,Price:$195 / night,Entire house in Nashville,SUPERHOST,/rooms/30266531?adults=10&check_in=2020-12-18&...,12 guests · 3 bedrooms · 10 beds · 2.5 baths,Price:$645 totalShow details,Rating 4.78 out of 5;4.7840 reviews (40)Price:...
...,...,...,...,...,...,...,...,...
275,"Sunset View Cottage, Nashville TN.",Price:$279 / night,Entire house in Nashville,,/rooms/11671995?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 1.5 baths,Price:$779 totalShow details,Rating 4.74 out of 5;4.7494 reviews (94)Price:...
276,Unwind On The SANITIZED Rooftop in Luxury,Previous price:$488Discounted price:$426 / night,Entire house in Nashville,SUPERHOST,/rooms/16080524?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 6 beds · 4 baths,"Price:$1,199 totalShow details",Rating 4.92 out of 5;4.9263 reviews (63)Previo...
277,Brand New Build / 2 Miles to Downtown,Price:$490 / night,Entire house in Nashville,SUPERHOST,/rooms/38693715?adults=10&check_in=2020-12-18&...,10 guests · 3 bedrooms · 5 beds · 4 baths,"Price:$1,341 totalShow details",Rating 4.93 out of 5;4.9330 reviews (30)Price:...
278,Group listing for 2 units! 1 Mile from Broadway!,Price:$326 / night,Entire apartment in Nashville,SUPERHOST,/rooms/42973154?adults=10&check_in=2020-12-18&...,12 guests · 4 bedrooms · 6 beds · 2 baths,Price:$972 totalShow details,Price:$326 / nightPrice:$972 totalShow details


In [7]:
first_search = page_data.append(search_data)
first_search.shape

(300, 8)